In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import your CRUD module
from crud import CRUD

###########################
# Data Manipulation / Model
###########################
# Update with your username and password and CRUD Python module name.
# Add additional parameters for host, port, database, and collection
username = "aacuser"
password = "SNHUCS340"
host = "nv-desktop-services.apporto.com"
port = 30441 
database = "AAC"
collection = "animals"

# Initialize the CRUD object with all necessary parameters
db = CRUD(username, password, host, port, database, collection)

# Fetch all records from the database
df = pd.DataFrame.from_records(db.read({}))

# Remove the '_id' column as it causes issues with data tables
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo with a URL anchor tag to the client's home page
image_filename = 'GraziosoSalvareLogo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.A(href='http://www.snhu.edu', children=[
        html.Img(src=f'data:image/png;base64,{encoded_image}')
    ]),
    html.Div(id='unique-identifier', children='Created by Matthew R. Leclerc'),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
        row_selectable='single',  # Allow single row selection
        selected_rows=[],  # Start with no row selected
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update data table based on filter selection
@app.callback(Output('datatable-id', 'data'), [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'reset':
        filtered_df = df
    else:
        if filter_type == 'water':
            breeds = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
            filtered_df = df[(df['breed'].isin(breeds)) & (df['sex_upon_outcome'] == 'Intact Female') & (df['age_upon_outcome_in_weeks'].between(26, 156))]
        elif filter_type == 'mount':
            breeds = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
            filtered_df = df[(df['breed'].isin(breeds)) & (df['sex_upon_outcome'] == 'Intact Male') & (df['age_upon_outcome_in_weeks'].between(26, 156))]
        elif filter_type == 'disaster':
            breeds = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
            filtered_df = df[(df['breed'].isin(breeds)) & (df['sex_upon_outcome'] == 'Intact Male') & (df['age_upon_outcome_in_weeks'].between(20, 300))]
    
    return filtered_df.to_dict('records')

# Update the charts based on the data table view
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return html.Div("No data to display")
    
    fig = px.pie(dff, names='breed', title='Preferred Animals')
    return dcc.Graph(figure=fig)

# Highlight selected data table cells
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]

# Updated map function with error handling
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data available")]

    dff = pd.DataFrame.from_dict(viewData)

    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    if dff.empty or row >= len(dff):
        return [html.Div("No valid data available")]

    # Check if required columns exist
    required_columns = ['location_lat', 'location_long', 'breed', 'name']
    if not all(col in dff.columns for col in required_columns):
        return [html.Div("Required data columns are missing")]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
               dl.TileLayer(id="base-layer-id"),
               dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']],
                         children=[
                             dl.Tooltip(dff.iloc[row]['breed']),
                             dl.Popup([
                                 html.H1("Animal Name"),
                                 html.P(dff.iloc[row]['name'])
                             ])
                         ])
           ])
    ]

if __name__ == '__main__':
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:20085/
